# 2 ROOT Basics

In [1]:
%jsroot on

Now that you have installed ROOT, what iss this interactive shell thing you’re running ? It is like this: ROOT leads a double life. It has an interpreter for macros Cling that you can run from the command line or like other applications. But it is also an interactive shell that can evaluate arbitrary statements and expressions. This is extremely useful for debugging, quick hacking and testing. In the notebook environment you will have a similar prompt allowing you to run ROOT commands straight from your browser. Let us first have a look at some very simple examples.

## 2.1 Controlling ROOT
One more remark at this point: as every command you type into ROOT is usually interpreted by Cling, an “escape character” is needed to pass commands to ROOT directly. This character is the dot at the beginning of a line:

    root [1] .<command>
  
This is a selection of the most common commands.
        
| Command | Description |
|------|------|
| .q | quit root |
| `.!<OS_command>` | access the shell |
| .x `<filename>`| execute a macro |
| .L `<filename>`| load a macro |
| .L `<filename>+`| compile a macro |     

Use .help at the prompt to inspect the full list.

## 2.2 ROOT a as calculator

You can even use the ROOT interactive shell instead of a calculator by launching the ROOT interactive shell with the command:

> root

on your Linux box. The prompt should appear shortly. Below you will find some examples:

In [2]:
1+1

(int) 2


In [3]:
2*(4+2)/12.

(double) 1.0000000


In [4]:
sqrt(3.)

(double) 1.7320508


In [5]:
1>2

(bool) false


In [6]:
TMath::Pi()

(double) 3.1415927


In [7]:
TMath::Erf(.2)

(double) 0.22270259


Not bad. You can see that ROOT offers you the possibility not only to type in C++ statements, but also advanced mathematical functions, which live in the [TMath namespace](https://root.cern.ch/root/html524/TMath.html).

Now let’s do something more elaborated. A numerical example with the well known geometrical series:

In [8]:
double x=.5

(double) 0.50000000


In [9]:
int N=30

(int) 30


In [10]:
double geom_series=0

(double) 0.0000000


In [11]:
for (int i=0;i<N;++i)geom_series+=TMath::Power(x,i)

In [12]:
TMath::Abs(geom_series - (1-TMath::Power(x,N-1))/(1-x))

(double) 1.8626451e-09


Here we made a step forward. We even declared variables and used a for control structure. Note that there are some subtle differences between Cling and the standard C++ language. You do not need the “;” at the end of line in interactive mode – try the difference e.g. declare a different double like in the command above. (NOTE: In the notebook environment you need to re-run the kernel in order to re-declare a variable.)

## 2.3 Learn C++ at the ROOT prompt

Behind the ROOT prompt there is an interpreter based on a real compiler toolkit: [LLVM](http://llvm.org). It is therefore possible to exercise many features of C++ and the standard library. For example in the following snippet we define a lambda function, a vector and we sort it in different ways:

In [13]:
typedef std::vector<double> doubles ;
auto pVec = [](const doubles& v){for (auto&& x:v) cout << x << endl;};
doubles v{0,3,5,4,1,2};
pVec(v);

0
3
5
4
1
2


In [14]:
std::sort(v.begin(),v.end());
pVec(v);

0
1
2
3
4
5


Or, if you prefer random number generation:

In [15]:
/*external JS*/
std::default_random_engine generator;
std::normal_distribution<double> distribution(0.,1.);
distribution(generator);
std::cout << distribution(generator);

-1.08682

In [16]:
distribution(generator);
std::cout << distribution(generator);

-1.07519

In [17]:
distribution(generator);
std::cout << distribution(generator);

0.744836

# 3 ROOT Macros

You know how other books go on and on about programming fundamentals and finally work up to building a complete, working program? Let’s skip all that. In this guide, we will describe macros executed by the ROOT C++ interpreter Cling.

It is relatively easy to compile a macro, either as a pre-compiled library to load into ROOT, or as a stand-alone application, by adding some include statements for header file or some “dressing code” to any macro.

## 3.1 General Remarks on ROOT macros

If you have a number of lines which you were able to execute at the ROOT prompt, they can be turned into a ROOT macro by giving them a name which corresponds to the file name without extension. The general structure for a macro stored in file ```MacroName.C``` is:

```C++
void MacroName() {
        <          ...
          your lines of C++ code
                   ...             >
}
```

The macro is executed by typing:

```sh
> root MacroName.C
```

at the system prompt, or executed using ```Bash .x``` at the ROOT prompt.

```zsh 
> root 
root [0] .x MacroName.C
```

 Or it can be loaded into a ROOT session and then be executed by typing:
 
```sh
root [0].L MacroName.C
root [1] MacroName();
```
at the ROOT prompt. The notebook environment is very similar. You can run a macro by typing:
```sh
MacroName();
```
on a new notebook cell. Note that more than one macro can be loaded this way, as each macro has a unique name in the ROOT name space. A small set of options can help making your plot nicer.

```C++
gROOT->SetStyle("Plain");   // set plain TStyle
gStyle->SetOptStat(111111); // draw statistics on plots,
                            // (0) for no output
gStyle->SetOptFit(1111);    // draw fit results on plot,
                            // (0) for no ouput
gStyle->SetPalette(57);     // set color map
gStyle->SetOptTitle(0);     // suppress title box
```

Next, you should create a canvas for graphical output, with size, subdivisions and format suitable to your needs, see documentation of class ```TCanvas```:
```C++
TCanvas canvas_3_1("31Canvas","Title",0,0,900,400); 
canvas_3_1.Divide(2,1); 
canvas_3_1.cd(1); 
```

These parts of a well-written macro are pretty standard, and you should remember to include pieces of code like in the examples above to make sure your plots always look as you had intended.

Below, in section Interpretation and Compilation, some more code fragments will be shown, allowing you to use the system compiler to compile macros for more efficient execution, or turn macros into stand-alone applications linked against the ROOT libraries.

## 3.2 A more complete example

Let us now look at a rather complete example of a typical task in data analysis, a macro that constructs a graph with errors, fits a (linear) model to it and saves it as an image. To run this macro, simply type in the shell:

```
 > root macro1.C
```

The code is built around the ROOT class ```TGraphErrors```, which was already introduced previously. Have a look at it in the class reference guide, where you will also find further examples. The macro shown below uses additional classes, ```TF1``` to define a function, TCanvas to define size and properties of the window used for our plot, and ```TLegend``` to add a nice legend. For the moment, ignore the commented include statements for header files, they will only become important at the end in section Interpretation and Compilation.

In [ ]:
%jsroot on

In [ ]:
%%cpp -d

// Builds a graph with errors, displays it and saves it as
// image. First, include some header files
// (not necessary for Cling)

#include "TCanvas.h"
#include "TROOT.h"
#include "TGraphErrors.h"
#include "TF1.h"
#include "TLegend.h"
#include "TArrow.h"
#include "TLatex.h"
    
    
void macro3_2_1(){   //#1
    // The values and the errors on the Y axis
    const int n_points=10;
    double x_vals[n_points]=
            {1,2,3,4,5,6,7,8,9,10};
    double y_vals[n_points]=
            {6,12,14,20,22,24,35,45,44,53};
    double y_errs[n_points]=
            {5,5,4.7,4.5,4.2,5.1,2.9,4.1,4.8,5.43};
        // Instance of the graph
    //#2
    TGraphErrors graph(n_points,x_vals,y_vals,nullptr,y_errs);
    graph.SetTitle("Measurement XYZ;lenght in cm;Arb Units");

    // Make the plot estetically better
    //#3
    graph.SetMarkerStyle(kOpenCircle);
    graph.SetMarkerColor(kBlue);
    graph.SetLineColor(kBlue);

    // The canvas on which we'll draw the graph
    //#4
    auto  Canvas_3_2_1 = new TCanvas();

    // Draw the graph !
    //#5
    graph.DrawClone("APE");

    // Define a linear function
    //#6
    TF1 function_3_2_1("Linear law","[0]+x*[1]",.5,10.5);
    // Let's make the funcion line nicer
    //#7
    function_3_2_1.SetLineColor(kRed);  function_3_2_1.SetLineStyle(2);
    // Fit it to the graph and draw it
    //#8
    graph.Fit(&function_3_2_1);
    function_3_2_1.DrawClone("Same");

    // Build and Draw a legend
    //#9
    TLegend leg(.1,.7,.3,.9,"Lab. Lesson 1");
    leg.SetFillColor(0);
    graph.SetFillColor(0);
    leg.AddEntry(&graph,"Exp. Points");
    leg.AddEntry(&function_3_2_1,"Th. Law");
    leg.DrawClone("Same");

    // Draw an arrow on the canvas
    //#10
    TArrow arrow(8,8,6.2,23,0.02,"|>");
    arrow.SetLineWidth(2);
    arrow.DrawClone();

    // Add some text to the plot
    //#11
    TLatex text(8.2,7.5,"#splitline{Maximum}{Deviation}");
    text.DrawClone();

    /*this command will create a pdf file with the graph in the same folder. 
    If you want to use it you can uncoment it and comment the Draw command bellow.*/
    //#12
    
//     mycanvas->Print("graph_with_law.pdf");
    Canvas_3_2_1->Draw();
}

Let’s have a look at the obtained plot. Beautiful outcome for such a small bunch of lines, isn’t it ?

Your first plot containing data points, a fit of an analytical function, a legend and some additional information in the form of graphics primitives and text. A clear well formatted plot, is crucial to communicate the relevance of your results to the reader.

In [ ]:
macro3_2_1();

Let’s comment it in detail:

* **Point #1:** the name of the principal function (it plays the role of the “main” function in compiled programs) in the macro file. It has to be the same as the file name without extension.

* **Point #2:** instance of the TGraphErrors class. The constructor takes the number of points and the pointers to the arrays of x values, y values, x errors (in this case none, represented by the NULL pointer) and y errors. The second line defines the title of the graph and the titles of the two axes, separated by a “;”.

* **Point #3:** These three lines are rather intuitive right ? To understand better the enumerators for colours and styles see the reference for the TColor and TMarker classes.

* **Point #4:** the canvas object that will host the drawn objects. The “memory leak” is intentional, to make the object exist also out of the macro1 scope.

* **Point #5:** the method [DrawClone](https://root.cern.ch/doc/master/classTObject.html#a45d0875bf30660d0903a93d690ff9f7e) draws a clone of the object on the canvas. It has to be a clone, to survive after the scope of macro1, and be displayed on screen after the end of the macro execution. The string option “APE” stands for:

    * A imposes the drawing of the Axes.

    * P imposes the drawing of the graph’s markers.

    * E imposes the drawing of the graph’s error bars.

* **Point #6:** define a mathematical function. There are several ways to accomplish this, but in this case the constructor accepts the name of the function, the formula, and the function range.

* **Point #7:** maquillage. Try to give a look to the line styles at your disposal visiting the documentation of the TLine class.

* **Point #8:** fits the f function to the graph, observe that the pointer is passed. It is more interesting to look at the output on the screen to see the parameters values and other crucial information that we will learn to read at the end of this guide. The DrawClone comand tha follows draws the clone of the object on the canvas again. The “Same” option avoids the cancellation of the already drawn objects, in our case, the graph. The function f will be drawn using the same axis system defined by the previously drawn graph.

* **Point #9:** completes the plot with a legend, represented by a TLegend instance. The constructor takes the lower left and upper right corners coordinates with respect to the total size of the canvas, assumed to be 1, and the legend header string, as parameters. You can add to the legend the objects, previously drawn or not drawn, through the addEntry method. Observe how the legend is drawn at the end: looks familiar now, right ?

* **Point #10:** defines an arrow with a triangle on the right hand side, a thickness of 2 and draws it.

* **Point #11:** interpret a Latex string which hast its lower left corner located in the specified coordinate. The #splitline{}{} construct allows to store multiple lines in the same TLatex object.

* **Point #12:** save the canvas as image. The format is automatically inferred from the file extension (it could have been eps, gif, …).


## 3.3 Interpretation and Compilation
As you observed, up to now we heavily exploited the capabilities of ROOT for interpreting our code, more than compiling and then executing. This is sufficient for a wide range of applications, but you might have already asked yourself “how can this code be compiled in my prompt?”. There are two answers.

### 3.3.1 Compile a Macro with ACLiC

[ACLiC](https://root.cern.ch/compiling-your-code-also-known-aclic) will create for you a compiled dynamic library for your macro, without any effort from your side, except the insertion of the appropriate header files at the top of the code. In this example, they are already included. To generate an object library from the macro code, from inside the interpreter type (please note the “+”):

```sh
 root [1] .L macro1.C+

```
Once this operation is accomplished, the macro symbols will be available in memory and you will be able to execute it simply by calling from inside the interpreter:

```sh
root [2] macro1()
```

### 3.3.2 Compile a Macro with the Compiler

A plethora of excellent compilers are available, both free and commercial. We will refer to the [```GCC```](https://gcc.gnu.org/onlinedocs/) compiler in the following. In this case, you have to include the appropriate headers in the code and then exploit the root-config tool for the automatic settings of all the compiler flags. root-config is a script that comes with ROOT; it prints all flags and libraries needed to compile code and link it with the ROOT libraries. In order to make the code executable stand-alone, an entry point for the operating system is needed, in C++ this is the procedure ```int main();```. The easiest way to turn a ROOT macro code into a stand-alone application is to add the following “dressing code” at the end of the macro file. This defines the procedure main, the only purpose of which is to call your macro:

```C++
int main() {
  ExampleMacro();
  return 0;
}
```

To create a stand-alone program from a macro called ```ExampleMacro.C```, simply type

```sh
 > g++ -o ExampleMacro ExampleMacro.C 'root-config --cflags --libs'
```

and execute it by typing:

```sh
 > ./ExampleMacro
```

This procedure will, however, not give access to the ROOT graphics, as neither control of mouse or keyboard events nor access to the graphics windows of ROOT is available. If you want your stand-alone application have display graphics output and respond to mouse and keyboard, a slightly more complex piece of code can be used. In the example below, a macro ```ExampleMacro_GUI``` is executed by the ROOT class [```TApplication```](https://root.cern.ch/doc/master/classTApplication.html). As a additional feature, this code example offers access to parameters eventually passed to the program when started from the command line. Here is the code fragment:

# 4 ROOT Beginners’ FAQ

At this point of the guide, some basic questions could have already come to your mind. We will try to clarify some of them with further explanations in the following.

## 4.1 ROOT type declarations for basic data types

In the official ROOT documentation, you find special data types replacing the normal ones, e.g. ```Double_t```, ```Float_t``` or ```Int_t``` replacing the standard ```double```, ```float``` or ```int``` types. Using the ROOT types makes it easier to port code between platforms (64/32 bit) or operating systems (windows/Linux), as these types are mapped to suitable ones in the ROOT header files. If you want adaptive code of this type, use the ROOT type declarations. However, usually you do not need such adaptive code, and you can safely use the standard C type declarations for your private code, as we did and will do throughout this guide. If you intend to become a ROOT developer, however, you better stick to the official coding rules!

## 4.2 Configure ROOT at start-up

The behaviour of a ROOT session can be tailored with the options in the ```.rootrc``` file. Examples of the tunable parameters are the ones related to the operating and window system, to the fonts to be used and to the location of start-up files. At start-up, ROOT looks for a ```.rootrc``` file in the following order:

* ```./.rootrc //local directory```
* ```$HOME/.rootrc //user directory```
* ```$ROOTSYS/etc/system.rootrc //global ROOT directory```

If more than one ```.rootrc``` file is in the search paths above, the options are merged, with precedence local, user, global. The parsing and interpretation of this file is handled by the ROOT class TEnv. Have a look at its documentation if you need such rather advanced features. The file ```.rootrc``` defines the location of two rather important files inspected at start-up: ```rootalias.C``` and ```rootlogon.C```. They can contain code that needs to be loaded and executed at ROOT startup. ```rootalias.C``` is only loaded and best used to define some often used functions. rootlogon.C contains code that will be executed at startup: this file is extremely useful for example to pre-load a custom style for the plots created with ROOT. This is done most easily by creating a new TStyle object with your preferred settings, as described in the class reference guide, and then use the command ```gROOT->SetStyle("MyStyleName");``` to make this new style definition the default one. As an example, have a look in the file rootlogon.C coming with this tutorial. Another relevant file is ```rootlogoff.C``` that is called when the session is finished.

## 4.3 ROOT command history

Every command typed at the ROOT prompt is stored in the file ```.root_hist``` in your home directory. ROOT uses this file to allow for navigation in the command history with the up-arrow and down-arrow keys. It is also convenient to extract successful ROOT commands with the help of a text editor for use in your own macros.

## 4.4 ROOT Global Pointers

All global pointers in ROOT begin with a small “g”. Some of them were already implicitly introduced (for example in the section Configure ROOT at start-up). The most important among them are presented in the following:

* **[gROOT](http://root.cern.ch/root/htmldoc/TROOT.html):** the ```gROOT``` variable is the entry point to the ```ROOT``` system. Technically it is an instance of the ```TROOT``` class. Using the gROOT pointer one has access to basically every object created in a ROOT based program. The ```TROOT``` object is essentially a container of several lists pointing to the main ```ROOT``` objects.

* **[gStyle](http://root.cern.ch/root/htmldoc/TStyle.html):** By default ROOT creates a default style that can be accessed via the ```gStyle``` pointer. This class includes functions to set some of the following object attributes.

    * Canvas
    * Pad
    * Histogram axis
    * Lines
    * Fill areas
    * Text
    * Markers
    * Functions
    * Histogram Statistics and Titles
    * etc …

* **[gSystem](http://root.cern.ch/root/htmldoc/TSystem.html):** An instance of a base class defining a generic interface to the underlying Operating System, in our case ```TUnixSystem```.

* **[gInterpreter](http://root.cern.ch/htmldoc/html/TInterpreter.html):** The entry point for the ROOT interpreter. Technically an abstraction level over a singleton instance of ```TCling```.

At this point you have already learned quite a bit about some basic features of ROOT.

<p id="nav-felt" style="possition:relative; width:50%; float:left;"><a href="1-ROOT-introduction.ipynb">&lt;&lt; Previous - Chapter 1: ROOT Motivation and Introduction</a></p>
<p id="nav-right" style="possition:relative; width:45%; float:left; text-align:right;"><a href="3-ROOT-functions-histograms-graphs.ipynb">Next - Chapter 3: ROOT Functions, Histograms and Graphs &gt;&gt;</a></p>